TASK 1 -> Read the dataset and do data pre-processing

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
df=pd.read_csv('/content/drug200.csv')
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [ ]:
df.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

In [ ]:
df['Drug'].value_counts()

DrugY    91
drugX    54
drugA    23
drugC    16
drugB    16
Name: Drug, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
df['Sex']=LE.fit_transform(df['Sex'])
df['BP']=LE.fit_transform(df['BP'])
df['Cholesterol']=LE.fit_transform(df['Cholesterol'])

In [ ]:
#spilitting the data
X=df.iloc[:,:-1].values
Y=pd.get_dummies(df['Drug']).values

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.25,random_state=0)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((150, 5), (50, 5), (150, 5), (50, 5))

TASK 2 -> Build the ANN model with (input layer, min 3 hidden layers & output layer)

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=32, activation='relu',input_dim=X_train.shape[1]))
ann.add(tf.keras.layers.Dense(64, activation='relu'))
ann.add(tf.keras.layers.Dense(64, activation='relu'))
ann.add(tf.keras.layers.Dense(5, activation='softmax'))

In [ ]:
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(ann.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                192       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 6,789
Trainable params: 6,789
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
ann.fit(X_train,Y_train,epochs=10,batch_size=16,validation_data=(X_test,Y_test))

Epoch 1/10
10/10 [==============================] - 2s 32ms/step - loss: 1.8335 - accuracy: 0.2667 - val_loss: 1.1431 - val_accuracy: 0.5000
Epoch 2/10
10/10 [==============================] - 0s 7ms/step - loss: 1.3660 - accuracy: 0.4733 - val_loss: 1.1281 - val_accuracy: 0.6200
Epoch 3/10
10/10 [==============================] - 0s 8ms/step - loss: 1.2276 - accuracy: 0.5600 - val_loss: 1.0197 - val_accuracy: 0.6200
Epoch 4/10
10/10 [==============================] - 0s 7ms/step - loss: 1.0887 - accuracy: 0.5467 - val_loss: 1.0681 - val_accuracy: 0.6200
Epoch 5/10
10/10 [==============================] - 0s 8ms/step - loss: 1.0410 - accuracy: 0.5600 - val_loss: 0.9634 - val_accuracy: 0.6200
Epoch 6/10
10/10 [==============================] - 0s 7ms/step - loss: 1.0194 - accuracy: 0.5467 - val_loss: 1.0250 - val_accuracy: 0.5400
Epoch 7/10
10/10 [==============================] - 0s 7ms/step - loss: 1.0088 - accuracy: 0.5800 - val_loss: 0.9505 - val_accuracy: 0.6200
Epoch 8/10
10/10 [=

In [ ]:
ypred = ann.predict(X_test)

2/2 [==============================] - 0s 4ms/step


TASK 3 -> Test the model with random data

In [ ]:
random_data = np.array([[80, 'M', 'HIGH', 'HIGH', 14.020]])

In [ ]:
random_data[:, 1] = LE.fit_transform(random_data[:, 1])
random_data[:, 2] = LE.fit_transform(random_data[:, 2])
random_data[:, 3] = LE.fit_transform(random_data[:, 3])

In [ ]:
random_data = random_data.astype(float)
random_data

array([[80.  ,  0.  ,  0.  ,  0.  , 14.02]])

In [ ]:
print(ann.predict(random_data))

1/1 [==============================] - 0s 23ms/step
[[0.04286237 0.09513308 0.530046   0.12218194 0.20977664]]


In [ ]:
predicted_class = np.argmax(ann.predict(random_data), axis=1)
print(predicted_class)

1/1 [==============================] - 0s 29ms/step
[2]


In [ ]:
drug_labels = {0: 'drugA', 1: 'drugB', 2: 'drugC', 3: 'drugX', 4: 'drugY'}
predicted_drug = [drug_labels[label] for label in predicted_class]
print("Predicted Drug:", predicted_drug)

Predicted Drug: ['drugC']
